Author: Vince Cooper, UW

This notebook calculates the distance inside the ice edge. 

Currently it is set up to calculate distance from satellite observations.

In [ ]:
import numpy as np
import os
import pandas as pd
import xarray as xr

import matplotlib as mpl
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
# from matplotlib.colors import DivergingNorm
# import matplotlib.patches as patches
%matplotlib inline
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns; sns.set(color_codes=False)
import cmocean
# from windrose import WindroseAxes
import warnings

import cftime
import datetime


plt.rcParams['xtick.bottom'] = True # keep my tick marks
plt.rcParams['ytick.left'] = True
plt.rcParams['font.size'] = 18
# plt.rcParams['figure.figsize'] = 12,8
# mpl.rcParams['figure.dpi'] = 300 # activate for presentation quality

from sklearn.metrics.pairwise import haversine_distances

## this is a dummy grid that has the right conventions
grid = xr.open_dataset(
    '/glade/work/vcooper/grid_ref/sithick_SImon_CESM2_piControl_r1i1p1f1_gn_110001-120012.nc')

## circle boundary for plotting
# theta = np.linspace(0, 2*np.pi, 100)
# center, radius = [0.5, 0.5], 0.5
# verts = np.vstack([np.sin(theta), np.cos(theta)]).T
# circle = mpl.path.Path(verts * radius + center)

In [2]:
## Updated version to 15% ice concentration threshold
def icedistance(iceconc_input):
    # turn icefracs to numpy array
    icefracsnp = iceconc_input.values
    lats = iceconc_input.TLAT.values # cice version
    lons = iceconc_input.TLON.values # cice version
#     lats = iceconc_input.latitude.values # wavewatch version
#     lons = iceconc_input.longitude.values # wavewatch version


    # create array to hold the distances
    distances = icefracsnp.copy() # same size array as the evaluated data
    distances -= distances # make zeros or nan; we will keep these values for cells that don't need a calc

    
    ##### GET OPEN WATER -> WATER/ICE EDGE LOCATIONS #####
    
    # get all open water locations except at edge of domain to avoid computation breaking
    icefracsnp_noborder = icefracsnp[1:-1,1:-1] # exclude borders for open water checking neighbors
    locations_openw = np.transpose(np.where(icefracsnp_noborder<0.15))
    locations_openw += 1 # adjust indices for the border exclusion

    # create 4 arrays, each represents the offset of open water location in coords by 1 unit
    latp1 = np.append(locations_openw[:,0]+1,locations_openw[:,1]).reshape(locations_openw.shape,order='F')
    latm1 = np.append(locations_openw[:,0]-1,locations_openw[:,1]).reshape(locations_openw.shape,order='F')
    lonp1 = np.append(locations_openw[:,0],locations_openw[:,1]+1).reshape(locations_openw.shape,order='F')
    lonm1 = np.append(locations_openw[:,0],locations_openw[:,1]-1).reshape(locations_openw.shape,order='F')

    # get max icefrac of 4 neighbor cells at each open water cell
    iceneighbormax = np.nanmax(np.stack((icefracsnp[latp1[:,0],latp1[:,1]],
                                         icefracsnp[lonm1[:,0],lonm1[:,1]],
                                         icefracsnp[lonp1[:,0],lonp1[:,1]],
                                         icefracsnp[latm1[:,0],latm1[:,1]])),axis=0)

    # get index of the open water cells with ice neighbor>15% # these are values for which we will calc distance
    wateredge = locations_openw[np.where(iceneighbormax>0.15)]
    wateredgeT = wateredge.T
    wateredgelatlon = np.array([[lats[wateredgeT[0],wateredgeT[1]]],
                                [lons[wateredgeT[0],wateredgeT[1]]]]).squeeze().T # Nx2 matrix of lat,lon
    
    ##### CALCULATION OF DISTANCES #####
    
    # get all cell locations with ice > 15%
    icewhere = np.where(icefracsnp>0.15)
    icecells = np.transpose(icewhere) # index by array position
    icelatlon = np.array([[lats[icewhere]],
                          [lons[icewhere]]]).squeeze().T # Nx2 matrix of lat,lon

    # calculate minimum distance
    mindist = haversine_distances(np.deg2rad(icelatlon),
                                  np.deg2rad(wateredgelatlon)).min(axis=1)*6371000/1000 # x by Radius-earth for km
    
    icecellsT = np.transpose(icecells) # transpose for vectorized indexing
    distances[icecellsT[0],icecellsT[1]] = mindist # put mindist into each grid point


    return(distances)

In [6]:
satdir2020 = '/glade/campaign/univ/uwas0070/vcooper/waveice/BGEP_vtc_backup20210120/regrid_sat/2020download/'
sat_data = xr.open_dataset(satdir2020 + 'seaice_conc_daily_nh_2020_cicegrid.nc')

In [8]:
%%time

## select which data array
ice_conc_data = sat_data.seaice_conc_cdr

## load 2012-2019 version of matching data
dist_sat = xr.open_dataarray(
    '/glade/work/vcooper/BGEP_vtc/sat_iceedgedistance_2012-2019.nc').rename('cdr')#.to_dataset()
# dist_sat_bt = xr.open_dataarray(
#     '/glade/work/vcooper/BGEP_vtc/sat_iceedgedistance_bt_2012-2019.nc').rename('bt').to_dataset()
# dist_sat_nt = xr.open_dataarray(
#     '/glade/work/vcooper/BGEP_vtc/sat_iceedgedistance_nt_2012-2019.nc').rename('nt').to_dataset()
# dist_sat_all = xr.merge([dist_sat,dist_sat_bt,dist_sat_nt])

## initialize distances array
distances = np.zeros(ice_conc_data.shape)

with warnings.catch_warnings():
    warnings.simplefilter("ignore") ## ignore warnings from div by zero leading to nans

    ## loop through time, calculating distance from ice edge
    for i in range(len(distances)):
        distances[i] = icedistance(ice_conc_data[i])

## convert to xarray and drop lower lats
distances_da = xr.DataArray(distances, dims=ice_conc_data.dims,coords=ice_conc_data.coords)
distances_da = distances_da.sel(nj=slice(300,384))

## combine with 2012-2019 data
dist_sat = xr.concat([dist_sat, distances_da],dim='time')

## save netcdf
savepath='/glade/campaign/univ/uwas0070/vcooper/waveice/icedistance/'
fname = 'sat_iceedgedistance_2012-2020.nc'
print ('saving to ', savepath + fname)

dist_sat.to_netcdf(path=savepath + fname)
print ('finished saving')

saving to  /glade/campaign/univ/uwas0070/vcooper/waveice/icedistance/sat_iceedgedistance_2012-2020.nc
finished saving
CPU times: user 1min 2s, sys: 1.03 s, total: 1min 3s
Wall time: 1min 5s


In [13]:
print(savepath + fname)
sat_data

/glade/campaign/univ/uwas0070/vcooper/waveice/icedistance/sat_iceedgedistance_2012-2020.nc


<xarray.Dataset>
Dimensions:                 (ni: 320, nj: 384, time: 366)
Coordinates:
  * time                    (time) datetime64[ns] 2020-01-01 ... 2020-12-31
    TLON                    (nj, ni) float32 ...
    TLAT                    (nj, ni) float32 ...
Dimensions without coordinates: ni, nj
Data variables:
    seaice_conc_cdr         (time, nj, ni) float32 ...
    goddard_nt_seaice_conc  (time, nj, ni) float32 ...
    goddard_bt_seaice_conc  (time, nj, ni) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.9 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Sun Jan 02 17:03:21 2022: cdo remapbil,cicegrid_dummy.nc 20...
    CDO:          Climate Data Operators version 1.9.9 (https://mpimet.mpg.de...